##Download the model from Roboflow

In [ ]:
from inference import get_model
#from google.colab import userdata

ROBOFLOW_API_KEY = "ITfUpuY5QO9WTBpcEXTh"  # Replace with your actual Roboflow API key

GOALPOST_DETECTION_MODEL_ID = "football-goalpost/3"
GOAPOSTL_DETECTION_MODEL = get_model(model_id=GOALPOST_DETECTION_MODEL_ID, api_key=ROBOFLOW_API_KEY)

##Use the model to detect the goalpost

In [ ]:
import supervision as sv
import numpy as np  

SOURCE_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/videos/goalMatch.mp4"

box_annotator = sv.BoxAnnotator(
    color=sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)

frame_number = 2000  # Change to the desired frame number
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

for i, frame in enumerate(frame_generator):
    if i == frame_number:
        # Run inference on the specific frame
        result = GOALPOST_DETECTION_MODEL.infer(frame, confidence=0.5)[0]
        detections = sv.Detections.from_inference(result)

        label_annotator = sv.LabelAnnotator(##labeling the box and adding text
          color = sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
          text_color = sv.Color.from_hex('#000000')
           )
        # Annotate the frame
        annotated_frame = frame.copy()
        annotated_frame = box_annotator.annotate(
            scene=annotated_frame,
            detections=detections
          )

        labels = [
           f"{class_name} {confidence:.2f}"
          for class_name, confidence
          in zip(detections["class_name"] , detections.confidence)
           ]

        annotated_frame = label_annotator.annotate(annotated_frame , detections,labels =labels)##annotate the frame with the labels
 
        # Show the annotated frame
        sv.plot_image(annotated_frame)


##Drwing a red circle on the goalpost center


In [ ]:
import cv2
import numpy as np
import supervision as sv
goalpost_center = None  # Variable to store the center of the goalpost

image_path = "C:/Users/user/Desktop/FootballAI/videos/image.png"  # Path to the image
image = cv2.imread(image_path)  # Load the image using OpenCV

result = GOALPOST_DETECTION_MODEL.infer(image, confidence=0.3)[0]

detections = sv.Detections.from_inference(result)
print(detections.class_id)

for detection, class_id in zip(detections.xyxy, detections.class_id):
    x_min, y_min, x_max, y_max = detection
    goalpost_center = ((x_min + x_max) / 2, (y_min + y_max) / 2)  # Calculate the center of the goalpost
    print(f"Goalpost center: {goalpost_center}")  # Print the center coordinates
    
    # Draw a red dotted circle at the center
    center_coordinates = (int(goalpost_center[0]), int(goalpost_center[1]))
    radius = 5
    color = (0, 0, 255)  # Red color in BGR
    thickness = 2
    line_type = cv2.LINE_AA  # Anti-aliased line for better quality
    cv2.circle(image, center_coordinates, radius, color, thickness, lineType=line_type)
    
    break  # Stop after finding the first goalpost

# Display the image with the red dotted circle
cv2.imshow("Image with Center Point", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
